# Documentation

This analysis tool reads .igc files from an indicated directory, filters the data using the `DataAnalyzer` helper class and then analyzes the speed data by applying the `SpeedAnalyzer` algorithm.

Inputs:
- INPUT_DIRECTORY: path to the directory containing .igc files
- OUTPUT_DIRECTORY: output data will be exported to this directory
- FILE_EXTENSION: set to `.igc`

# Imports

In [ ]:
import os
import sys
import pandas as pd
from typing import List, Tuple
from datetime import datetime

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

import constants as constants
import helpers.file_processor as file_processor
import helpers.data_visualizer as data_visualizer
import algorithms.speed_analyzer as speed_analyzer

# Variables

In [ ]:
INPUT_DIRECTORY: str = "/Users/nicolas/Dropbox/3_Bildung/2_KZO/Stichwortverzeichnis/0 Maturarbeit/6 Wettbewerbe/1_SJf/3_Coaching/2_Projekt/data-analysis/1_tracklogs/2_analysis-data/1_raw/2_glide"
OUTPUT_DIRECTORY: str = "/Users/nicolas/Downloads/" # end with /
FILE_EXTENSION: str = ".igc"

# Initialisation

In [ ]:
file_processor = file_processor.FileProcessor()
speed_analyzer = speed_analyzer.SpeedAnalyzer()
data_visualizer = data_visualizer.DataVisualizer()

theoretical_reference: pd.DataFrame = pd.read_csv(f"{constants.THEORETICAL_REFERENCE_PATH}")
original_reference: pd.DataFrame = pd.read_csv(f"{constants.ORIGINAL_REFERENCE_PATH}")

timestamp: str = datetime.now().strftime("%Y%m%d-%H%M%S")

# File Listing

In [ ]:
file_paths: List[str] = file_processor.get_file_paths(path=INPUT_DIRECTORY, file_extension=FILE_EXTENSION)

In [ ]:
print(f"Importing files for the following conditions:")
print(f"--> Directory: {INPUT_DIRECTORY}")
print(f"--> File extension: {FILE_EXTENSION}")
print(f"--> Theoretical polar: {constants.THEORETICAL_REFERENCE_PATH.split("/")[-1]}")
print(f"--> Original reference: {constants.ORIGINAL_REFERENCE_PATH.split("/")[-1]}")
print()
print("Files:")
print(f"--> Found {len(file_paths)} files.")
print(f"--> The processing is initiated.")

# File Processing

In [ ]:
data_raw: pd.DataFrame = speed_analyzer.process_raw_data(file_paths=file_paths)

# Data Processing

In [ ]:
theoretical_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=theoretical_reference, reference=True)
original_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=original_reference, reference=True)
data_raw_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=data_raw)

print(f"Filtered data:")
print(f"--> Data points for filtered theoretical reference: {len(theoretical_reference_filtered)} (lost {len(theoretical_reference) - len(theoretical_reference_filtered)})")
print(f"--> Data points for filtered original reference: {len(original_reference_filtered)} (lost {len(original_reference) - len(original_reference_filtered)})")
print(f"--> Data points for filtered tracklogs: {len(data_raw_filtered)} (lost {len(data_raw) - len(data_raw_filtered)})")
print()
print(f"Please note:")
print(f"--> Why are so many raw data points lost? This is due to the fact that the system filters out all data point that are not on a straight line. This is done to ensure that the data is as accurate as possible.")

smoothed_data_raw: pd.DataFrame = speed_analyzer.savgol_filter(data=data_raw_filtered)
smoothed_data_grouped: pd.DataFrame = speed_analyzer.group_data(data=smoothed_data_raw)

print(f"Data smoothing:")
print(f"--> During smoothing, the raw data points were reduced from {len(data_raw_filtered)} to {len(smoothed_data_raw)} (lost {len(data_raw_filtered) - len(smoothed_data_raw)}).")
print(f"--> During grouping, the smoothed data points were reduced from {len(smoothed_data_raw)} to {len(smoothed_data_grouped)} (lost {len(smoothed_data_raw) - len(smoothed_data_grouped)}).")
print()
print(f"Export data to CSV:")
print(f"--> Exporting smoothed data to {timestamp}_SJf_smoothed-experimental-speed-data_nicolas-huber.csv.")
print(f"--> Exporting grouped data to {timestamp}_SJf_experimental-speed-data_nicolas-huber.csv.")

speed_analyzer.export_to_csv(data=smoothed_data_raw, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_smoothed-experimental-speed-data_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=smoothed_data_grouped, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_grouped-experimental-speed-data_nicolas-huber.csv")


# Visualisation

In [ ]:
print(f"Data visualization:")
print(f"--> For reference, the theoretical and experimental speed data is visualized.")
print(f"--> To compare a older version of this algorithm, please refer to the following two plots:")
print()

data_visualizer.visualize_raw_speed_data(experimental_data=original_reference_filtered, theoretical_data=theoretical_reference_filtered, std_error=0.2, title="Experimentelle Geschwindigkeistdaten (original)")
data_visualizer.visualize_raw_speed_data(experimental_data=smoothed_data_grouped, theoretical_data=theoretical_reference_filtered, std_error=0.05, title="Experimentelle Geschwindigkeistdaten (optimiert)")

print(f"--> To visualize the quality increase of the data, the following two plots are shown:")

deviation_original: Tuple[float, float, float, float] = data_visualizer.visualize_speed_deviation(experimental_data=original_reference_filtered, theoretical_data=theoretical_reference_filtered, speed_analyzer=speed_analyzer, title="Abweichung der experimentellen Geschwindigkeitsdaten (original)")
deviation_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_speed_deviation(experimental_data=smoothed_data_grouped, theoretical_data=theoretical_reference_filtered, speed_analyzer=speed_analyzer, title="Abweichung der experimentellen Geschwindigkeitsdaten (optimiert)")

# Report

In [ ]:
score_original: float = speed_analyzer.score_stats(deviation_original)
score_optimized: float = speed_analyzer.score_stats(deviation_optimized)

print(f"Report:")
speed_analyzer.print_report(score_original=score_original, score_optimized=score_optimized, deviation_original=deviation_original, deviation_optimized=deviation_optimized, datasets=[original_reference, original_reference_filtered, data_raw, smoothed_data_grouped])

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")